# توجہ کے میکانزم اور ٹرانسفارمرز

ریکرنٹ نیٹ ورکس کا ایک بڑا نقص یہ ہے کہ کسی ترتیب میں تمام الفاظ کا نتیجے پر یکساں اثر ہوتا ہے۔ اس کی وجہ سے نامیاتی اداروں کی شناخت (Named Entity Recognition) اور مشین ترجمہ جیسے ترتیب سے ترتیب کے کاموں کے لیے معیاری LSTM انکوڈر-ڈیکوڈر ماڈلز کی کارکردگی کمزور ہو جاتی ہے۔ حقیقت میں، ان پٹ ترتیب کے مخصوص الفاظ اکثر ترتیب وار نتائج پر دوسروں کے مقابلے میں زیادہ اثر ڈالتے ہیں۔

ایک ترتیب سے ترتیب ماڈل پر غور کریں، جیسے مشین ترجمہ۔ یہ دو ریکرنٹ نیٹ ورکس کے ذریعے نافذ کیا جاتا ہے، جہاں ایک نیٹ ورک (**انکوڈر**) ان پٹ ترتیب کو ایک چھپی ہوئی حالت میں تبدیل کرتا ہے، اور دوسرا نیٹ ورک، **ڈیکوڈر**، اس چھپی ہوئی حالت کو ترجمہ شدہ نتیجے میں کھولتا ہے۔ اس طریقہ کار کے ساتھ مسئلہ یہ ہے کہ نیٹ ورک کی آخری حالت کو جملے کے آغاز کو یاد رکھنے میں مشکل پیش آتی ہے، جس کی وجہ سے لمبے جملوں پر ماڈل کی کارکردگی خراب ہو جاتی ہے۔

**توجہ کے میکانزم** ہر ان پٹ ویکٹر کے سیاق و سباق کے اثر کو RNN کے ہر آؤٹ پٹ پیش گوئی پر وزن دینے کا ایک ذریعہ فراہم کرتے ہیں۔ اس کو نافذ کرنے کا طریقہ یہ ہے کہ ان پٹ RNN کی درمیانی حالتوں اور آؤٹ پٹ RNN کے درمیان شارٹ کٹس بنائے جائیں۔ اس طرح، جب آؤٹ پٹ علامت $y_t$ پیدا کی جا رہی ہو، تو ہم تمام ان پٹ چھپی ہوئی حالتوں $h_i$ کو مختلف وزن کے گتانک $\alpha_{t,i}$ کے ساتھ مدنظر رکھیں گے۔

![ایک انکوڈر/ڈیکوڈر ماڈل کی تصویر جس میں ایک اضافی توجہ کی تہہ شامل ہے](../../../../../translated_images/ur/encoder-decoder-attention.7a726296894fb567.webp)
*انکوڈر-ڈیکوڈر ماڈل اضافی توجہ کے میکانزم کے ساتھ [Bahdanau et al., 2015](https://arxiv.org/pdf/1409.0473.pdf) سے لیا گیا، [اس بلاگ پوسٹ](https://lilianweng.github.io/lil-log/2018/06/24/attention-attention.html) سے حوالہ شدہ*

توجہ میٹرکس $\{\alpha_{i,j}\}$ اس حد کو ظاہر کرے گی کہ ان پٹ ترتیب کے کون سے الفاظ آؤٹ پٹ ترتیب میں دیے گئے لفظ کی تخلیق میں کردار ادا کرتے ہیں۔ نیچے ایک ایسی میٹرکس کی مثال دی گئی ہے:

![ایک نمونہ ترتیب کی تصویر جو RNNsearch-50 کے ذریعے پائی گئی، Bahdanau - arviz.org سے لی گئی](../../../../../translated_images/ur/bahdanau-fig3.09ba2d37f202a6af.webp)

*[Bahdanau et al., 2015](https://arxiv.org/pdf/1409.0473.pdf) (شکل 3) سے لی گئی تصویر*

توجہ کے میکانزم موجودہ یا قریب موجودہ قدرتی زبان کی پروسیسنگ میں بہترین کارکردگی کے لیے ذمہ دار ہیں۔ تاہم، توجہ شامل کرنے سے ماڈل کے پیرامیٹرز کی تعداد میں بہت زیادہ اضافہ ہوتا ہے، جس کی وجہ سے RNNs کے ساتھ اسکیلنگ کے مسائل پیدا ہوتے ہیں۔ RNNs کو اسکیل کرنے کی ایک اہم رکاوٹ یہ ہے کہ ماڈلز کی ریکرنٹ نوعیت تربیت کو بیچ اور متوازی بنانے میں چیلنج پیدا کرتی ہے۔ RNN میں ترتیب کے ہر عنصر کو ترتیب وار عمل کرنا ضروری ہوتا ہے، جس کا مطلب ہے کہ اسے آسانی سے متوازی نہیں بنایا جا سکتا۔

توجہ کے میکانزم کو اپنانے اور اس رکاوٹ کے امتزاج نے ان جدید ترین ٹرانسفارمر ماڈلز کی تخلیق کی راہ ہموار کی، جنہیں ہم آج BERT سے لے کر OpenGPT3 تک استعمال کرتے ہیں۔

## ٹرانسفارمر ماڈلز

پچھلی پیش گوئی کے سیاق و سباق کو اگلے جائزہ مرحلے میں منتقل کرنے کے بجائے، **ٹرانسفارمر ماڈلز** **پوزیشنل انکوڈنگز** اور **توجہ** کا استعمال کرتے ہیں تاکہ دیے گئے ان پٹ کے سیاق و سباق کو فراہم کردہ متن کی ونڈو کے اندر قید کیا جا سکے۔ نیچے دی گئی تصویر دکھاتی ہے کہ کس طرح پوزیشنل انکوڈنگز کے ساتھ توجہ ایک دی گئی ونڈو کے اندر سیاق و سباق کو قید کر سکتی ہے۔

![ایک متحرک GIF جو دکھاتا ہے کہ ٹرانسفارمر ماڈلز میں جائزے کیسے کیے جاتے ہیں۔](../../../../../lessons/5-NLP/18-Transformers/images/transformer-animated-explanation.gif)

چونکہ ہر ان پٹ پوزیشن کو آزادانہ طور پر ہر آؤٹ پٹ پوزیشن پر نقشہ بنایا جاتا ہے، ٹرانسفارمرز RNNs کے مقابلے میں بہتر متوازی بن سکتے ہیں، جو کہ بہت بڑے اور زیادہ اظہار کرنے والے زبان کے ماڈلز کو قابل بناتا ہے۔ ہر توجہ کا سر مختلف الفاظ کے درمیان تعلقات سیکھنے کے لیے استعمال کیا جا سکتا ہے، جو نیچے دیے گئے قدرتی زبان کی پروسیسنگ کے کاموں کو بہتر بناتا ہے۔

## سادہ ٹرانسفارمر ماڈل بنانا

Keras میں بلٹ ان ٹرانسفارمر لیئر شامل نہیں ہے، لیکن ہم اپنا خود کا بنا سکتے ہیں۔ جیسا کہ پہلے، ہم AG News ڈیٹاسیٹ کے متن کی درجہ بندی پر توجہ مرکوز کریں گے، لیکن یہ ذکر کرنا ضروری ہے کہ ٹرانسفارمر ماڈلز زیادہ مشکل NLP کاموں میں بہترین نتائج دکھاتے ہیں۔


In [1]:
import tensorflow as tf
from tensorflow import keras
import tensorflow_datasets as tfds
import numpy as np

ds_train, ds_test = tfds.load('ag_news_subset').values()

def extract_text(x):
    return x['title']+' '+x['description']

def tupelize(x):
    return (extract_text(x),x['label'])

نئے لیئرز کو کیراس میں `Layer` کلاس کو سب کلاس کرنا چاہیے، اور `call` میتھڈ کو نافذ کرنا چاہیے۔ آئیے **پوزیشنل ایمبیڈنگ** لیئر سے شروع کرتے ہیں۔ ہم [کیراس کی آفیشل دستاویزات سے کچھ کوڈ](https://keras.io/examples/nlp/text_classification_with_transformer/) استعمال کریں گے۔ ہم فرض کریں گے کہ ہم تمام ان پٹ سیکوینسز کو `maxlen` لمبائی تک پیڈ کرتے ہیں۔


In [2]:
class TokenAndPositionEmbedding(keras.layers.Layer):
    def __init__(self, maxlen, vocab_size, embed_dim):
        super(TokenAndPositionEmbedding, self).__init__()
        self.token_emb = keras.layers.Embedding(input_dim=vocab_size, output_dim=embed_dim)
        self.pos_emb = keras.layers.Embedding(input_dim=maxlen, output_dim=embed_dim)
        self.maxlen = maxlen

    def call(self, x):
        maxlen = self.maxlen
        positions = tf.range(start=0, limit=maxlen, delta=1)
        positions = self.pos_emb(positions)
        x = self.token_emb(x)
        return x+positions

یہ پرت دو `Embedding` پرتوں پر مشتمل ہے: ایک ٹوکنز کو ایمبیڈ کرنے کے لیے (جس طریقے پر ہم پہلے بات کر چکے ہیں) اور دوسری ٹوکن پوزیشنز کے لیے۔ ٹوکن پوزیشنز کو قدرتی نمبروں کی ترتیب کے طور پر 0 سے `maxlen` تک `tf.range` کا استعمال کرتے ہوئے بنایا جاتا ہے، اور پھر انہیں ایمبیڈنگ پرت کے ذریعے گزارا جاتا ہے۔ نتیجے میں بننے والے دو ایمبیڈنگ ویکٹرز کو جمع کیا جاتا ہے، جس سے ان پٹ کا پوزیشنلی ایمبیڈڈ نمائندگی پیدا ہوتی ہے جس کی شکل `maxlen`$\times$`embed_dim` ہوتی ہے۔

اب، آئیے ٹرانسفارمر بلاک کو نافذ کرتے ہیں۔ یہ پہلے سے بیان کردہ ایمبیڈنگ پرت کے آؤٹ پٹ کو لے گا:


In [3]:
class TransformerBlock(keras.layers.Layer):
    def __init__(self, embed_dim, num_heads, ff_dim, rate=0.1):
        super(TransformerBlock, self).__init__()
        self.att = keras.layers.MultiHeadAttention(num_heads=num_heads, key_dim=embed_dim, name='attn')
        self.ffn = keras.Sequential(
            [keras.layers.Dense(ff_dim, activation="relu"), keras.layers.Dense(embed_dim),]
        )
        self.layernorm1 = keras.layers.LayerNormalization(epsilon=1e-6)
        self.layernorm2 = keras.layers.LayerNormalization(epsilon=1e-6)
        self.dropout1 = keras.layers.Dropout(rate)
        self.dropout2 = keras.layers.Dropout(rate)

    def call(self, inputs, training):
        attn_output = self.att(inputs, inputs)
        attn_output = self.dropout1(attn_output, training=training)
        out1 = self.layernorm1(inputs + attn_output)
        ffn_output = self.ffn(out1)
        ffn_output = self.dropout2(ffn_output, training=training)
        return self.layernorm2(out1 + ffn_output)

اب ہم مکمل ٹرانسفارمر ماڈل کی تعریف کرنے کے لیے تیار ہیں:


In [4]:
embed_dim = 32  # Embedding size for each token
num_heads = 2  # Number of attention heads
ff_dim = 32  # Hidden layer size in feed forward network inside transformer
maxlen = 256
vocab_size = 20000

model = keras.models.Sequential([
    keras.layers.experimental.preprocessing.TextVectorization(max_tokens=vocab_size,output_sequence_length=maxlen, input_shape=(1,)),
    TokenAndPositionEmbedding(maxlen, vocab_size, embed_dim),
    TransformerBlock(embed_dim, num_heads, ff_dim),
    keras.layers.GlobalAveragePooling1D(),
    keras.layers.Dropout(0.1),
    keras.layers.Dense(20, activation="relu"),
    keras.layers.Dropout(0.1),
    keras.layers.Dense(4, activation="softmax")
])

model.summary()

Model: "sequential_1"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
text_vectorization (TextVect (None, 256)               0         
_________________________________________________________________
token_and_position_embedding (None, 256, 32)           648192    
_________________________________________________________________
transformer_block (Transform (None, 256, 32)           10656     
_________________________________________________________________
global_average_pooling1d (Gl (None, 32)                0         
_________________________________________________________________
dropout_2 (Dropout)          (None, 32)                0         
_________________________________________________________________
dense_2 (Dense)              (None, 20)                660       
_________________________________________________________________
dropout_3 (Dropout)          (None, 20)               

In [5]:
print('Training tokenizer')
model.layers[0].adapt(ds_train.map(extract_text))
model.compile(loss='sparse_categorical_crossentropy',metrics=['acc'], optimizer='adam')
model.fit(ds_train.map(tupelize).batch(128),validation_data=ds_test.map(tupelize).batch(128))

Training tokenizer
938/938 [==============================] - 45s 39ms/step - loss: 0.4978 - acc: 0.8068 - val_loss: 0.2808 - val_acc: 0.9124


## بی ای آر ٹی ٹرانسفارمر ماڈلز

**بی ای آر ٹی** (Bidirectional Encoder Representations from Transformers) ایک بہت بڑا ملٹی لیئر ٹرانسفارمر نیٹ ورک ہے جس میں *BERT-base* کے لیے 12 لیئرز اور *BERT-large* کے لیے 24 لیئرز ہیں۔ یہ ماڈل پہلے بڑی مقدار میں ٹیکسٹ ڈیٹا (ویکیپیڈیا + کتابیں) پر غیر نگرانی شدہ تربیت کے ذریعے (جملے میں ماسک کیے گئے الفاظ کی پیش گوئی کرتے ہوئے) تربیت یافتہ ہوتا ہے۔ تربیت کے دوران ماڈل زبان کو سمجھنے کی ایک اہم سطح حاصل کرتا ہے، جسے پھر دیگر ڈیٹاسیٹس کے ساتھ فائن ٹیوننگ کے ذریعے استعمال کیا جا سکتا ہے۔ اس عمل کو **ٹرانسفر لرننگ** کہا جاتا ہے۔

![تصویر http://jalammar.github.io/illustrated-bert/ سے](../../../../../translated_images/ur/jalammarBERT-language-modeling-masked-lm.34f113ea5fec4362.webp)

ٹرانسفارمر آرکیٹیکچرز کی کئی اقسام ہیں، جن میں بی ای آر ٹی، ڈسٹل بی ای آر ٹی، بگ برڈ، اوپن جی پی ٹی 3 اور مزید شامل ہیں، جنہیں فائن ٹیون کیا جا سکتا ہے۔

آئیے دیکھتے ہیں کہ ہم پہلے سے تربیت یافتہ بی ای آر ٹی ماڈل کو اپنے روایتی سیکوئنس کلاسیفیکیشن مسئلے کو حل کرنے کے لیے کیسے استعمال کر سکتے ہیں۔ ہم [سرکاری دستاویزات](https://www.tensorflow.org/text/tutorials/classify_text_with_bert) سے کچھ آئیڈیاز اور کوڈ لیں گے۔

پہلے سے تربیت یافتہ ماڈلز کو لوڈ کرنے کے لیے، ہم **ٹینسر فلو حب** استعمال کریں گے۔ سب سے پہلے، آئیے بی ای آر ٹی کے مخصوص ویکٹرائزر کو لوڈ کرتے ہیں:


In [1]:
import tensorflow_text 
import tensorflow_hub as hub
vectorizer = hub.KerasLayer('https://tfhub.dev/tensorflow/bert_en_uncased_preprocess/3')

ModuleNotFoundError: No module named 'tensorflow_text'

In [7]:
vectorizer(['I love transformers'])

{'input_type_ids': <tf.Tensor: shape=(1, 128), dtype=int32, numpy=
 array([[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
         0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
         0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
         0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
         0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
         0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0]],
       dtype=int32)>,
 'input_word_ids': <tf.Tensor: shape=(1, 128), dtype=int32, numpy=
 array([[  101,  1045,  2293, 19081,   102,     0,     0,     0,     0,
             0,     0,     0,     0,     0,     0,     0,     0,     0,
             0,     0,     0,     0,     0,     0,     0,     0,     0,
             0,     0,     0,     0,     0,     0,     0,     0,     0,
             0,     0,     0,     0,     0,     0,     0,     0,     0,
             0,     0,     0,     0,     0, 

یہ ضروری ہے کہ آپ وہی ویکٹرائزر استعمال کریں جو اصل نیٹ ورک کی تربیت کے لیے استعمال ہوا تھا۔ مزید یہ کہ، BERT ویکٹرائزر تین اجزاء واپس کرتا ہے:
* `input_word_ids`، جو ان پٹ جملے کے لیے ٹوکن نمبرز کی ترتیب ہے
* `input_mask`، جو دکھاتا ہے کہ ترتیب کا کون سا حصہ اصل ان پٹ پر مشتمل ہے اور کون سا پیڈنگ ہے۔ یہ `Masking` لیئر کے ذریعے بننے والے ماسک سے ملتا جلتا ہے
* `input_type_ids` زبان ماڈلنگ کے کاموں کے لیے استعمال ہوتا ہے، اور ایک ترتیب میں دو ان پٹ جملے بتانے کی اجازت دیتا ہے۔

اس کے بعد، ہم BERT فیچر ایکسٹریکٹر کو انسٹیٹیوٹ کر سکتے ہیں:


In [8]:
bert = hub.KerasLayer('https://tfhub.dev/tensorflow/small_bert/bert_en_uncased_L-4_H-128_A-2/1')

In [9]:
z = bert(vectorizer(['I love transformers']))
for i,x in z.items():
    print(f"{i} -> { len(x) if isinstance(x, list) else x.shape }")

pooled_output -> (1, 128)
encoder_outputs -> 4
sequence_output -> (1, 128, 128)
default -> (1, 128)


تو، BERT لیئر کئی مفید نتائج فراہم کرتی ہے:
* `pooled_output` تمام ٹوکنز کو سیکوئنس میں اوسط کرنے کا نتیجہ ہے۔ آپ اسے پورے نیٹ ورک کی ایک ذہین معنوی ایمبیڈنگ کے طور پر دیکھ سکتے ہیں۔ یہ ہمارے پچھلے ماڈل میں `GlobalAveragePooling1D` لیئر کے آؤٹ پٹ کے برابر ہے۔
* `sequence_output` آخری ٹرانسفارمر لیئر کا آؤٹ پٹ ہے (جو ہمارے اوپر والے ماڈل میں `TransformerBlock` کے آؤٹ پٹ کے مطابق ہے)۔
* `encoder_outputs` تمام ٹرانسفارمر لیئرز کے آؤٹ پٹس ہیں۔ چونکہ ہم نے 4-لیئر BERT ماڈل لوڈ کیا ہے (جیسا کہ آپ شاید نام سے اندازہ لگا سکتے ہیں، جس میں `4_H` شامل ہے)، اس میں 4 ٹینسرز ہیں۔ آخری ٹینسر `sequence_output` کے برابر ہے۔

اب ہم اختتام سے اختتام تک کلاسیفکیشن ماڈل کی تعریف کریں گے۔ ہم *فنکشنل ماڈل ڈیفینیشن* استعمال کریں گے، جہاں ہم ماڈل کا ان پٹ ڈیفائن کریں گے، اور پھر اس کے آؤٹ پٹ کو کیلکولیٹ کرنے کے لیے ایک سلسلہ وار ایکسپریشنز فراہم کریں گے۔ ہم BERT ماڈل کے ویٹس کو نان-ٹرین ایبل بنائیں گے، اور صرف آخری کلاسیفائر کو ٹرین کریں گے:


In [10]:
inp = keras.Input(shape=(),dtype=tf.string)
x = vectorizer(inp)
x = bert(x)
x = keras.layers.Dropout(0.1)(x['pooled_output'])
out = keras.layers.Dense(4,activation='softmax')(x)
model = keras.models.Model(inp,out)
bert.trainable = False
model.summary()

Model: "model"
__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
input_1 (InputLayer)            [(None,)]            0                                            
__________________________________________________________________________________________________
keras_layer (KerasLayer)        {'input_type_ids': ( 0           input_1[0][0]                    
__________________________________________________________________________________________________
keras_layer_1 (KerasLayer)      {'pooled_output': (N 4782465     keras_layer[0][0]                
                                                                 keras_layer[0][1]                
                                                                 keras_layer[0][2]                
______________________________________________________________________________________________

In [11]:
model.compile(loss='sparse_categorical_crossentropy',metrics=['acc'], optimizer='adam')
model.fit(ds_train.map(tupelize).batch(128),validation_data=ds_test.map(tupelize).batch(128))

938/938 [==============================] - 528s 559ms/step - loss: 0.8056 - acc: 0.6983 - val_loss: 0.5953 - val_acc: 0.7888


اگرچہ تربیت کے قابل پیرامیٹرز کم ہیں، لیکن یہ عمل کافی سست ہے کیونکہ BERT فیچر ایکسٹریکٹر حسابی طور پر بھاری ہے۔ ایسا لگتا ہے کہ ہم مناسب درستگی حاصل کرنے میں ناکام رہے، یا تو تربیت کی کمی کی وجہ سے، یا ماڈل کے پیرامیٹرز کی کمی کی وجہ سے۔

آئیے BERT کے وزن کو غیر منجمد کرنے اور اسے بھی تربیت دینے کی کوشش کرتے ہیں۔ اس کے لیے بہت کم لرننگ ریٹ کی ضرورت ہوتی ہے، اور ساتھ ہی زیادہ محتاط تربیتی حکمت عملی کی ضرورت ہوتی ہے جس میں **وارم اپ** شامل ہو، اور **AdamW** آپٹیمائزر کا استعمال کیا جائے۔ ہم `tf-models-official` پیکج کا استعمال کرتے ہوئے آپٹیمائزر بنائیں گے:


In [12]:
from official.nlp import optimization 
bert.trainable=True
model.summary()
epochs = 3
opt = optimization.create_optimizer(
    init_lr=3e-5,
    num_train_steps=epochs*len(ds_train),
    num_warmup_steps=0.1*epochs*len(ds_train),
    optimizer_type='adamw')

model.compile(loss='sparse_categorical_crossentropy',metrics=['acc'], optimizer=opt)
model.fit(ds_train.map(tupelize).batch(128),validation_data=ds_test.map(tupelize).batch(128))

Model: "model"
__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
input_1 (InputLayer)            [(None,)]            0                                            
__________________________________________________________________________________________________
keras_layer (KerasLayer)        {'input_type_ids': ( 0           input_1[0][0]                    
__________________________________________________________________________________________________
keras_layer_1 (KerasLayer)      {'pooled_output': (N 4782465     keras_layer[0][0]                
                                                                 keras_layer[0][1]                
                                                                 keras_layer[0][2]                
______________________________________________________________________________________________

جیسا کہ آپ دیکھ سکتے ہیں، تربیت کافی آہستہ چلتی ہے - لیکن آپ تجربہ کرنا چاہیں گے اور ماڈل کو چند ایپکس (5-10) کے لیے تربیت دیں اور دیکھیں کہ آیا آپ ان طریقوں کے مقابلے میں بہترین نتیجہ حاصل کر سکتے ہیں جو ہم نے پہلے استعمال کیے ہیں۔

## Huggingface Transformers لائبریری

Transformer ماڈلز استعمال کرنے کا ایک اور عام (اور تھوڑا آسان) طریقہ [HuggingFace پیکیج](https://github.com/huggingface/) ہے، جو مختلف NLP کاموں کے لیے سادہ بلڈنگ بلاکس فراہم کرتا ہے۔ یہ Tensorflow اور PyTorch دونوں کے لیے دستیاب ہے، جو ایک اور بہت مشہور نیورل نیٹ ورک فریم ورک ہے۔

> **نوٹ**: اگر آپ یہ دیکھنے میں دلچسپی نہیں رکھتے کہ Transformers لائبریری کیسے کام کرتی ہے - تو آپ اس نوٹ بک کے آخر تک جا سکتے ہیں، کیونکہ آپ کو اوپر کیے گئے کاموں سے کچھ خاص مختلف نظر نہیں آئے گا۔ ہم BERT ماڈل کو تربیت دینے کے وہی مراحل دہرا رہے ہوں گے، لیکن ایک مختلف لائبریری اور کافی بڑے ماڈل کے ساتھ۔ لہذا، اس عمل میں کچھ کافی طویل تربیت شامل ہے، تو آپ صرف کوڈ کو دیکھنے پر غور کر سکتے ہیں۔

آئیے دیکھتے ہیں کہ ہمارا مسئلہ [Huggingface Transformers](http://huggingface.co) استعمال کرتے ہوئے کیسے حل ہو سکتا ہے۔


سب سے پہلے ہمیں یہ فیصلہ کرنا ہوگا کہ ہم کون سا ماڈل استعمال کریں گے۔ کچھ بلٹ ان ماڈلز کے علاوہ، Huggingface میں ایک [آن لائن ماڈل ریپوزٹری](https://huggingface.co/models) موجود ہے، جہاں آپ کمیونٹی کے ذریعے فراہم کردہ مزید پہلے سے تربیت یافتہ ماڈلز تلاش کر سکتے ہیں۔ ان تمام ماڈلز کو صرف ماڈل کا نام فراہم کر کے لوڈ اور استعمال کیا جا سکتا ہے۔ ماڈل کے لیے درکار تمام بائنری فائلز خود بخود ڈاؤن لوڈ ہو جائیں گی۔

کبھی کبھار آپ کو اپنے ماڈلز لوڈ کرنے کی ضرورت پیش آ سکتی ہے، ایسی صورت میں آپ اس ڈائریکٹری کو مخصوص کر سکتے ہیں جو تمام متعلقہ فائلز پر مشتمل ہو، جیسے کہ ٹوکنائزر کے پیرامیٹرز، `config.json` فائل جس میں ماڈل کے پیرامیٹرز ہوں، بائنری ویٹس وغیرہ۔

ماڈل کے نام سے، ہم ماڈل اور ٹوکنائزر دونوں کو انسٹیٹیوٹ کر سکتے ہیں۔ آئیے ٹوکنائزر سے شروع کرتے ہیں:


In [2]:
import transformers

# To load the model from Internet repository using model name. 
# Use this if you are running from your own copy of the notebooks
bert_model = 'bert-base-uncased' 

# To load the model from the directory on disk. Use this for Microsoft Learn module, because we have
# prepared all required files for you.
#bert_model = './bert'

tokenizer = transformers.BertTokenizer.from_pretrained(bert_model)

MAX_SEQ_LEN = 128
PAD_INDEX = tokenizer.convert_tokens_to_ids(tokenizer.pad_token)
UNK_INDEX = tokenizer.convert_tokens_to_ids(tokenizer.unk_token)

`tokenizer` کا آبجیکٹ `encode` فنکشن پر مشتمل ہے جو متن کو براہ راست انکوڈ کرنے کے لیے استعمال کیا جا سکتا ہے۔


In [3]:
tokenizer.encode('Tensorflow is a great framework for NLP')

[101, 23435, 12314, 2003, 1037, 2307, 7705, 2005, 17953, 2361, 102]

ہم ٹوکنائزر کو اس طرح استعمال کر سکتے ہیں کہ ایک سلسلہ ماڈل کو دینے کے لیے موزوں ہو، یعنی `token_ids`، `input_mask` فیلڈز وغیرہ شامل ہوں۔ ہم یہ بھی وضاحت کر سکتے ہیں کہ ہم Tensorflow ٹینسرز چاہتے ہیں، `return_tensors='tf'` دلیل فراہم کر کے:


In [4]:
tokenizer(['Hello, there'],return_tensors='tf')

{'input_ids': <tf.Tensor: shape=(1, 5), dtype=int32, numpy=array([[ 101, 7592, 1010, 2045,  102]], dtype=int32)>, 'token_type_ids': <tf.Tensor: shape=(1, 5), dtype=int32, numpy=array([[0, 0, 0, 0, 0]], dtype=int32)>, 'attention_mask': <tf.Tensor: shape=(1, 5), dtype=int32, numpy=array([[1, 1, 1, 1, 1]], dtype=int32)>}

ہم اس معاملے میں پہلے سے تربیت یافتہ BERT ماڈل `bert-base-uncased` استعمال کریں گے۔ *Uncased* کا مطلب ہے کہ یہ ماڈل کیس-انسینسیٹو ہے۔

جب ماڈل کو تربیت دی جا رہی ہو، تو ہمیں ٹوکنائزڈ سیکوئنس کو ان پٹ کے طور پر فراہم کرنا ہوگا، اور اسی لیے ہم ڈیٹا پروسیسنگ پائپ لائن تیار کریں گے۔ چونکہ `tokenizer.encode` ایک Python فنکشن ہے، ہم پچھلے یونٹ کی طرح ہی طریقہ استعمال کریں گے اور اسے `py_function` کے ذریعے کال کریں گے:


In [31]:
def process(x):
    return tokenizer.encode(x.numpy().decode('utf-8'),return_tensors='tf',padding='max_length',max_length=MAX_SEQ_LEN,truncation=True)[0]

def process_fn(x):
    s = x['title']+' '+x['description']
    e = tf.py_function(process,inp=[s],Tout=(tf.int32))
    e.set_shape(MAX_SEQ_LEN)
    return e,x['label']

اب ہم `BertForSequenceClassfication` پیکج کا استعمال کرتے ہوئے اصل ماڈل لوڈ کر سکتے ہیں۔ یہ یقینی بناتا ہے کہ ہمارے ماڈل میں پہلے سے ہی درجہ بندی کے لیے مطلوبہ فن تعمیر موجود ہے، بشمول آخری کلاسیفائر۔ آپ کو ایک انتباہی پیغام نظر آئے گا جس میں کہا جائے گا کہ آخری کلاسیفائر کے وزن شروع نہیں کیے گئے ہیں، اور ماڈل کو پری ٹریننگ کی ضرورت ہوگی - یہ بالکل ٹھیک ہے، کیونکہ یہی وہ کام ہے جو ہم کرنے والے ہیں!


In [32]:
model = transformers.TFBertForSequenceClassification.from_pretrained(bert_model,num_labels=4,output_attentions=False)

In [33]:
model.summary()

Model: "tf_bert_for_sequence_classification_1"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
bert (TFBertMainLayer)       multiple                  109482240 
_________________________________________________________________
dropout_75 (Dropout)         multiple                  0         
_________________________________________________________________
classifier (Dense)           multiple                  3076      
Total params: 109,485,316
Trainable params: 109,485,316
Non-trainable params: 0
_________________________________________________________________


جیسا کہ آپ `summary()` سے دیکھ سکتے ہیں، ماڈل میں تقریباً 110 ملین پیرامیٹرز شامل ہیں! غالباً، اگر ہم نسبتاً چھوٹے ڈیٹاسیٹ پر سادہ درجہ بندی کا کام چاہتے ہیں، تو ہم BERT بیس لیئر کو ٹرین نہیں کرنا چاہتے:


In [34]:
model.layers[0].trainable = False
model.summary()

Model: "tf_bert_for_sequence_classification_1"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
bert (TFBertMainLayer)       multiple                  109482240 
_________________________________________________________________
dropout_75 (Dropout)         multiple                  0         
_________________________________________________________________
classifier (Dense)           multiple                  3076      
Total params: 109,485,316
Trainable params: 3,076
Non-trainable params: 109,482,240
_________________________________________________________________


اب ہم تربیت شروع کرنے کے لیے تیار ہیں!

> **نوٹ**: مکمل پیمانے پر BERT ماڈل کی تربیت میں بہت زیادہ وقت لگ سکتا ہے! اس لیے ہم اسے صرف پہلے 32 بیچز کے لیے تربیت دیں گے۔ یہ صرف یہ دکھانے کے لیے ہے کہ ماڈل کی تربیت کیسے ترتیب دی جاتی ہے۔ اگر آپ مکمل پیمانے پر تربیت آزمانا چاہتے ہیں - تو `steps_per_epoch` اور `validation_steps` پیرامیٹرز کو ہٹا دیں، اور انتظار کرنے کے لیے تیار ہو جائیں!


In [30]:
model.compile('adam','sparse_categorical_crossentropy',['acc'])
tf.get_logger().setLevel('ERROR')
model.fit(ds_train.map(process_fn).batch(32),validation_data=ds_test.map(process_fn).batch(32),steps_per_epoch=32,validation_steps=2)

32/32 [==============================] - 142s 4s/step - loss: 1.3896 - acc: 0.2500 - val_loss: 1.3863 - val_acc: 0.2480


اگر آپ تکرار کی تعداد بڑھائیں، زیادہ وقت انتظار کریں، اور کئی ادوار کے لیے تربیت کریں، تو آپ توقع کر سکتے ہیں کہ BERT کی درجہ بندی ہمیں بہترین درستگی فراہم کرے گی! اس کی وجہ یہ ہے کہ BERT پہلے ہی زبان کے ڈھانچے کو کافی حد تک سمجھتا ہے، اور ہمیں صرف آخری درجہ بندی کو بہتر بنانا ہوتا ہے۔ تاہم، چونکہ BERT ایک بڑا ماڈل ہے، اس لیے پورا تربیتی عمل کافی وقت لیتا ہے اور زبردست کمپیوٹیشنل طاقت کی ضرورت ہوتی ہے! (GPU، اور ترجیحاً ایک سے زیادہ)۔

> **نوٹ:** ہماری مثال میں، ہم نے سب سے چھوٹے پہلے سے تربیت یافتہ BERT ماڈلز میں سے ایک استعمال کیا ہے۔ بڑے ماڈلز موجود ہیں جو ممکنہ طور پر بہتر نتائج فراہم کر سکتے ہیں۔


## خلاصہ

اس یونٹ میں، ہم نے **ٹرانسفارمرز** پر مبنی جدید ترین ماڈل آرکیٹیکچرز کا جائزہ لیا۔ ہم نے انہیں اپنے ٹیکسٹ کلاسیفکیشن کے کام کے لیے استعمال کیا، لیکن اسی طرح، BERT ماڈلز کو اینٹیٹی ایکسٹریکشن، سوالات کے جوابات دینے، اور دیگر NLP کاموں کے لیے بھی استعمال کیا جا سکتا ہے۔

ٹرانسفارمر ماڈلز NLP میں موجودہ جدید ترین تکنیک کی نمائندگی کرتے ہیں، اور زیادہ تر معاملات میں یہ وہ پہلی حل ہونا چاہیے جس کے ساتھ آپ اپنی کسٹم NLP حلوں پر تجربہ کرنا شروع کریں۔ تاہم، اگر آپ جدید نیورل ماڈلز بنانا چاہتے ہیں تو اس ماڈیول میں زیر بحث ری کرنٹ نیورل نیٹ ورکس کے بنیادی اصولوں کو سمجھنا انتہائی اہم ہے۔



---

**ڈسکلیمر**:  
یہ دستاویز AI ترجمہ سروس [Co-op Translator](https://github.com/Azure/co-op-translator) کا استعمال کرتے ہوئے ترجمہ کی گئی ہے۔ ہم درستگی کے لیے کوشش کرتے ہیں، لیکن براہ کرم آگاہ رہیں کہ خودکار ترجمے میں غلطیاں یا غیر درستیاں ہو سکتی ہیں۔ اصل دستاویز کو اس کی اصل زبان میں مستند ذریعہ سمجھا جانا چاہیے۔ اہم معلومات کے لیے، پیشہ ور انسانی ترجمہ کی سفارش کی جاتی ہے۔ ہم اس ترجمے کے استعمال سے پیدا ہونے والی کسی بھی غلط فہمی یا غلط تشریح کے ذمہ دار نہیں ہیں۔
